Escreva uma função de "limpeza" que recebe o nome de um arquivo CSV como entrada e retorna um DataFrame.

A função deve realizar os seguintes passos:

- Selecione os dados no arquivo CSV e retorne apenas apartamentos na Cidade do México ("Distrito Federal") que custem menos de $100.000.

- Remova outliers cortando os 10% inferiores e superiores das propriedades em termos de "surface_covered_in_m2".

- Crie colunas separadas "lat" e "lon".

- A Cidade do México é dividida em 15 bairros. Crie uma feature "borough" a partir da coluna "place_with_parent_names".

- Exclua colunas que contenham mais de 50% de valores nulos.

- Exclua colunas que contenham valores categóricos de baixa ou alta cardinalidade.

- Exclua quaisquer colunas que constituam vazamento para o alvo "price_aprox_usd".

- Exclua quaisquer colunas que criariam problemas de multicolinearidade

Use o glob para criar a lista de arquivos. Ela deve conter os nomes dos arquivos de todos os CSVs de imóveis da Cidade do México no diretório ./data, exceto o mexico-city-features.csv.

Combine sua função de "limpeza", uma compreensão de lista e pd.concat para criar um DataFrame df.

Ele deve conter todas as propriedades dos cinco arquivos CSV em files.

Crie um histograma mostrando a distribuição dos preços de apartamentos ("price_aprox_usd") em df.

Certifique-se de rotular o eixo x como "Price [$]", o eixo y como "Count" e dar o título "Distribution of Apartment Prices". Use o Matplotlib (plt).

Como é a distribuição dos preços? Os dados são normais, ligeiramente assimétricos ou muito assimétricos?

Crie um gráfico de dispersão que mostre o preço dos apartamentos ("price_aprox_usd") como uma função do tamanho dos apartamentos ("surface_covered_in_m2").

Certifique-se de rotular o eixo x como "Area [sq meters]" e o eixo y como "Price [USD]". Seu gráfico deve ter o título "Mexico City: Price vs. Area".

Use o Matplotlib (plt).

Você vê alguma relação entre preço e área nos dados? Como isso é semelhante ou diferente do conjunto de dados de Buenos Aires?

Crie um gráfico de dispersão no Mapbox que mostre a localização dos apartamentos em seu conjunto de dados e represente seu preço usando cores.

Quais áreas da cidade parecem ter preços de imóveis mais altos?

Crie sua matriz de características X_train e o vetor alvo y_train.

Seu alvo é "price_aprox_usd". Suas características devem ser todas as colunas que restam no DataFrame que você limpou acima.

Calcule o erro absoluto médio de referência para o seu modelo.

Crie um pipeline chamado model que contenha todos os transformadores necessários para este conjunto de dados e um dos preditores que você usou durante este projeto. Em seguida, ajuste seu modelo aos dados de treinamento.

Leia o arquivo CSV mexico-city-features.csv para o DataFrame X_test.

Use seu modelo para gerar uma Série de previsões para X_test. Quando você enviar suas previsões para o avaliador, ele calculará o erro absoluto médio para o seu modelo.

Crie um `Serie` chamado feat_imp. O índice deve conter os nomes de todas as características que seu modelo considera ao fazer previsões;

Os valores devem ser os valores dos coeficientes associados a cada característica.

A Série deve ser ordenada em ordem crescente pelo valor absoluto.

Crie um gráfico de barras horizontal que mostre os 10 coeficientes mais influentes para seu modelo.

Certifique-se de rotular o eixo x como "Importance [USD]" e o eixo y como "Feature", e dê ao seu gráfico o título "Feature Importances for Apartment Price". Use o pandas.